# Data acquisition in arbok

### Arbok module imports

In [1]:
from arbok.samples.aurora.configs.aurora_qua_config import aurora_qua_config

from arbok.samples.aurora.initialization.odd_6_dot_init import InitOdd6dot
from arbok.samples.aurora.initialization.Tminus_6_dot_init import InitTminus6dot
from arbok.samples.aurora.readout.psb_6_dot_read import PSB6dotRead

2023-09-15 13:46:21,574 - qm - INFO     - Starting session: 282a1aca-f435-4907-9277-a77f866257c6


In [2]:
from arbok.core.program import Program
from arbok.core.sample import Sample

Instanciating sample and measurement objects

In [3]:
aurora = Sample('aurora', aurora_qua_config)
MEAS = Program('ESR_search', sample = aurora)

Creating Subsequences and adding them to the measurement object `MEAS`

In [4]:
init_odd = InitOdd6dot('init_odd')
read1 = PSB6dotRead('read1')
init_Tminus = InitTminus6dot('init_Tminus')
read2 = PSB6dotRead('read2')

In [5]:
MEAS.add_subsequence(init_odd)
MEAS.add_subsequence(read1)
MEAS.add_subsequence(init_Tminus)
MEAS.add_subsequence(read2)

Next we connect the physical OPX+ instrument to our program ... 

In [6]:
MEAS.connect_OPX(host_ip= '192.168.0.3')

2023-09-15 13:46:22,233 - qm - INFO     - Performing health check
2023-09-15 13:46:22,282 - qm - INFO     - Health check passed
2023-09-15 13:46:25,030 - qm - WARNING  - Open QM ended with warning 0: Opening a new Quantum Machine and closing existing identical Quantum Machine qm-1694750174527
2023-09-15 13:46:25,031 - qm - WARNING  - Open QM ended with warning 0: Quantum Machine qm-1694750174527 is canceling running job 1684735462124.


In [7]:
import numpy as np

In [17]:
load_ramp_time_list =np.arange(4,1000, 40, dtype = int)
init_load_mixed_list =np.arange(4,1000, 40, dtype = int)
print(np.shape(load_ramp_time_list), np.shape(init_load_mixed_list))

(25,) (25,)


Defining the parameters to be swept and adding setpoints to them. Also parameters to be recorded are added. In this case the measured state of SET 2

In [9]:
"""
For parameter sweep along two axises -> get() will return n x m shaped array (n )
"""
MEAS.settables = [
    MEAS.init_odd.tDeltaLoadRampMixed,
    MEAS.init_odd.tInitLoadMixed
]
MEAS.setpoints_grid = [
    load_ramp_time_list, init_load_mixed_list
]

In [10]:
""" 
For parameter sweep along ONE axises while adjusting all parameters at the same 
time -> get() will return n shaped array (n: length of both input arrays -> 
needs to be the same)
"""
MEAS.settables = [
    [MEAS.init_odd.tDeltaLoadRampMixed,
    MEAS.init_Tminus.tInitLoadMixed,]
]

MEAS.setpoints_grid = [
    [init_load_mixed_list,init_load_mixed_list],
]

' \nFor parameter sweep along ONE axises while adjusting all parameters at the same \ntime -> get() will return n shaped array (n: length of both input arrays -> \nneeds to be the same)\n'

Just defined values are automatically added to `meas_ctrl` by running `prepare_meas_ctrl`

... and run the job! Dont worry, the program will only run the first iteration and wait for us to retreive the result and only continue after that.

In [11]:
MEAS.run()

Adding qua <class 'int'> variable for tDeltaLoadRampMixed
Adding qua <class 'int'> variable for tInitLoadMixed
Adding qua sweep loop for tInitLoadMixed
Adding qua sweep loop for tDeltaLoadRampMixed
2023-09-15 13:46:26,020 - qm - INFO     - Sending program to QOP for compilation
2023-09-15 13:46:39,887 - qm - INFO     - Executing program


After the system is compiled successfully you can start streaming results.
Note however that you have to stop and run the sequence again with new parameters for different setpoints.

In [ ]:
MEAS.read1.diff_set2_read_I.get()